In [138]:
import pandas as pd
import numpy as np

from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.aflow_prototypes import AflowPrototypeMatcher
from pymatgen.analysis.aflow_prototypes import AFLOW_PROTOTYPE_LIBRARY
from pymatgen.analysis.local_env import CrystalNN

from matminer.featurizers.structure import SiteStatsFingerprint

### Generate database of mineral fingerprints

In [79]:
data = {'structure': [], 'mineral': [],
        'pearson': [], 'aflow': [],
        'strukturbericht': []}

for d in AFLOW_PROTOTYPE_LIBRARY:
    data['structure'].append(d['snl'].structure)
    data['mineral'].append(d['tags']['mineral'])
    data['pearson'].append(d['tags']['pearson'])
    data['aflow'].append(d['tags']['aflow'])
    data['strukturbericht'].append(d['tags']['strukturbericht'])

df = pd.DataFrame(data).set_index('aflow')

In [122]:
ssf = SiteStatsFingerprint.from_preset('CrystalNNFingerprint_ops', stats=('mean', 'std_dev'))

df = ssf.featurize_dataframe(df, 'structure', ignore_errors=True, return_errors=True)

# put finger print in its own column
df['fingerprint'] = df[ssf.feature_labels()].values.tolist()
df = df.drop(ssf.feature_labels(), axis=1)

# drop failed rows and exceptions column
df = df[pd.isnull(df['SiteStatsFingerprint Exceptions'])]
df = df.drop(['SiteStatsFingerprint Exceptions'], axis=1)

minerals = df[df['mineral'] != '']

print("database contains {} minerals".format(len(minerals)))

In [199]:
minerals['ntypesp'] = minerals['structure'].apply(lambda x: x.ntypesp)

In [200]:
from matminer.utils.io import store_dataframe_as_json

store_dataframe_as_json(minerals, 'mineral_dataframe.json.gz', compression='gz')

### Download some structures

In [154]:
mpr = MPRester()

data = mpr.get_entry_by_material_id('mp-856', inc_structure='final')
rutile = data.structure

data = mpr.get_entry_by_material_id('mp-2534', inc_structure='final')
gaas = data.structure

data = mpr.get_entry_by_material_id('mp-4019', inc_structure='final')
catio = data.structure

data = mpr.get_entry_by_material_id('mp-23514', inc_structure='final')
bsi = data.structure

data = mpr.get_entry_by_material_id('mp-27636', inc_structure='final')
csi = data.structure

### Test mineral matching by AflowPrototype Matcher

In [174]:
def get_aflow_match(struct):
    formula = struct.composition.reduced_formula
    
    matcher = AflowPrototypeMatcher()
    mineral_match = matcher.get_prototypes(struct)
    
    if mineral_match:
        print("{} is {}-structured".format(formula, mineral_match[0]['tags']['mineral']))
    else:
        print("Could not determine structure of {}".format(formula))

get_aflow_match(csi)

Could not determine structure of Cs2SnI6


### Test custom mineral matching using site fingerprint

In [180]:
def get_closest_fingerprint(struct, n_prototypes=3,
                            match_ntypesp=True):
    ssf = SiteStatsFingerprint.from_preset('CrystalNNFingerprint_ops', stats=('mean', 'std_dev'))
    fingerprint = np.array(ssf.featurize(struct))
    
    def get_distance(x_struct, x_fingerprint):
        dist = np.linalg.norm(x_fingerprint - fingerprint)
        
        # bad match if number species different
        if match_ntypesp:
            sp_diff = abs(x_struct.ntypesp - struct.ntypesp) * 1000
        else:
            sp_diff = 0
        return dist + sp_diff
    
    X = minerals.copy()
    X['distance'] = minerals[['structure', 'fingerprint']].apply(
        lambda x: get_distance(*x), axis=1)
    X = X.sort_values(by='distance')
    
    prototypes = [{'mineral': getattr(x, 'mineral'), 
                   'distance': getattr(x, "distance")}
                  for x in list(X.itertuples())[:n_prototypes]]
    
    return prototypes

In [175]:
get_closest_fingerprint(csi)

[{'mineral': 'Orthorhombic Perovskite', 'distance': 1.060991692109146},
 {'mineral': 'Bergman Structure: Mg32(Al,Zn)49 Bergman',
  'distance': 1.1175589396791292},
 {'mineral': 'Pb (Zr_0.50 Ti_0.48) O_3', 'distance': 1.1535256241846368}]

In [184]:
def get_guesses(mpid, match_ntypesp=True):
    mpr = MPRester()

    data = mpr.get_entry_by_material_id(mpid, inc_structure='final')
    struct = data.structure
    
    get_aflow_match(struct)
    
    display(get_closest_fingerprint(struct, match_ntypesp=match_ntypesp))

In [193]:
get_guesses('mp-2657', True)

TiO2 is Rutile-structured


[{'mineral': 'Rutile', 'distance': 0.02641798811622957},
 {'mineral': 'Hydrophilite', 'distance': 0.0487514575875521},
 {'mineral': 'beta Vanadium nitride', 'distance': 0.22563184557335536}]

### Site Descriptions

SnO2 is rutile-structured and crystallises in the P4nmm space group. It is a three-dimensionally connected structure. Sn is octahedrally coordinated to 6 symmetrically equivalent O atoms. With the oxygen in turn coordinated to three symmetrically equivalent Sn atoms.

From literature:


The rutile structure (Fig. 2) is composed of distorted cation octahedra interconnected by the sharing of corner and edge O atoms. Each octahedron is elongated along an axis passing through opposite corners, resulting in two
longer and four shorter cation-O bonds. The octahedra share edges to form octahedral chains, which run parallel
to the c axis. Within a chain, each octahedron is positioned with its long axis perpendicular to the length of the chain. Each chain is linked to four adjacent chains by sharing corner O atoms. Adjacent chains are shifted 1/2 c and rotated 90 deg around the c axis (chain axis) relative to each other. This arrangement leaves O in trigonal planar coordination with respect to cations, with one longer and two shorter cation-O bond distances. The 0-0 distance along the shared edge within the chain is much shorter than the unshared edges of the octahedra.

In [2]:
# function for converting structure to code

mpr = MPRester()

data = mpr.get_entry_by_material_id('mp-1001', inc_structure='final')
struct = data.structure
flatten = lambda l: [item for sublist in l for item in sublist]

print([float("{:.2f}".format(x)) for x in flatten(struct.lattice.matrix)])
print([s.name for s in struct.species])
print([[float("{:.2f}".format(x)) for x in coords] for coords in struct.frac_coords])

[4.16, -0.02, 0.94, 1.76, 3.77, 0.94, 0.01, 0.01, 7.34]
['N', 'N', 'N', 'N', 'Ba', 'Ba']
[[0.15, 0.45, 0.45], [0.55, 0.85, 0.05], [0.45, 0.15, 0.95], [0.85, 0.55, 0.55], [0.79, 0.21, 0.25], [0.21, 0.79, 0.75]]


In [222]:
ssf = SiteStatsFingerprint.from_preset('CrystalNNFingerprint_ops', stats=None)

features = ssf.featurize(struct)

In [226]:
from matminer.featurizers.site import CrystalNNFingerprint

cnnf = CrystalNNFingerprint.from_preset("ops")
finger = dict(zip(cnnf.feature_labels(), cnnf.featurize(struct, 0)))

In [140]:
from pymatgen.analysis.local_env import CrystalNN

cnn = CrystalNN()
#cnn.get_bonded_structure(rutile)

### Structure fragment identifier

In [139]:
mpr = MPRester()

data = mpr.get_entry_by_material_id('mp-22849', inc_structure='final')
bi = data.structure
bi_bonded = cnn.get_bonded_structure(bi)
bi_bonded

NameError: name 'cnn' is not defined

In [141]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

sga = SpacegroupAnalyzer(bi)
bi_sym = sga.get_symmetrized_structure()
bi_sym.equivalent_indices

[[0, 1, 2, 3, 4, 5], [6, 7]]

BiI3 description:
    
BiI3 is Bismuth triodide structured with the R-3 spacegroup.
It is a layered structure containing two BiI3 layers in the primitive cell.
In each BiI3 layer, I is bonded in an L-shaped geometry to two symmetrically equivalent Bi atoms.
Bi is bonded in an octahedral geometry to six symmetrically equivalent I atoms. In this arrangement, there are three shorter (3.10 Å) and three longer (3.13 Å) Bi–I bond lengths.


In [8]:
bi_bonded.draw_graph_to_file(filename='graph.png')

In [3]:
from pymatgen.analysis.structure_analyzer import get_dimensionality

def dimen(mp_id):
    mpr = MPRester()
    data = mpr.get_entry_by_material_id(mp_id, inc_structure='final')
    struct = data.structure
    return get_dimensionality(struct)
    
dimen('mp-22849')

2

### First attempt at fragment finder

In [98]:
visited_sites = set()

def get_connected_sites(site_id):
    for site in bi_bonded.get_connected_sites(site_id):
        coords = tuple([site.site.specie.name] + list(site.site.coords))
        if coords not in visited_sites:
            visited_sites.add(coords)
            if site.jimage == (0, 0, 0):
                get_connected_sites(site.index)

get_connected_sites(0)

In [99]:
visited_sites

{('Bi', 3.597648480628507, 2.4528338367059113, 15.43717319481898),
 ('Bi', 6.138333780628507, 8.926383896705913, 11.828199934818981),
 ('Bi', 6.764510589371494, 4.611962553294089, 13.03480508518102),
 ('Bi', 10.551922890628507, 2.452833766705911, 11.82819997481898),
 ('I', 3.7284661615606898, 3.914498653041081, 12.701611393202947),
 ('I', 6.248614494368066, 7.298300439760502, 14.502563801663667),
 ('I', 6.63369290843931, 3.1502977369589193, 15.770366886797053),
 ('I', 6.654229875631933, 6.240046010239498, 10.360441218336332),
 ('I', 7.504308736923883, 2.0406016254153276, 11.405129856124484),
 ('I', 9.812124743076119, 5.024194694584671, 13.457875203875517)}

#### Create fake structure with box around fragment

In [110]:
from pymatgen.core.structure import Structure
from pyobb.obb import OBB

box = OBB.build_from_points([x[1:] for x in visited_sites])

new_bi = Structure(
    [20, 0, 0, 0, 20, 0, 0, 0, 20], 
    [x[0] for x in visited_sites] + ['S'] * len(box.points), 
    [x[1:] for x in visited_sites] + box.points,
    coords_are_cartesian=True
)
new_bi.to(fmt='cif', filename='test.cif')

### Use fragment finger to evaluate dimensionality

In [156]:
from pymatgen.analysis.local_env import CrystalNN
cnn = CrystalNN()

def get_dimensionality(structure):
    bonded_struct = cnn.get_bonded_structure(structure)
    
    set_of_visited_sites = set()
    
    for i, site in enumerate(structure):
        visited_sites = set()
        def get_connected_sites(site_id):
            for site in bonded_struct.get_connected_sites(site_id):
                if site not in visited_sites:
                    visited_sites.add(site)
                    if site.jimage == (0, 0, 0):
                        get_connected_sites(site.index)
        get_connected_sites(i)
        set_of_visited_sites.add(frozenset(visited_sites))

    print("{} groups of connected sites".format(len(set_of_visited_sites)))
    
    n_images = [len(set([site.jimage for site in visited_sites])) for visited_sites in set_of_visited_sites]
    print("the sets have {} images".format(n_images))

In [168]:
get_dimensionality(bsi)

7 groups of connected sites
the sets have [4, 3, 3, 3, 4, 2, 2] images


Obviously this does not work when you have fragments that are composed from multiple images.

### Define functions to work out which supercell image a point belongs to

In [309]:
def point_in_lattice(lattice, point):
    u = np.cross(lattice[1], lattice[2])
    v = np.cross(lattice[0], lattice[2])
    w = np.cross(lattice[0], lattice[1])
    
    up = np.dot(u, point)
    vp = np.dot(v, point)
    wp = np.dot(w, point)
    
    ua = np.dot(u, lattice[0])
    vb = np.dot(v, lattice[1])
    wc = np.dot(w, lattice[2])

    if (up > min(0, ua) and up < max(0, ua) and
        vp > min(0, vb) and vp < max(0, vb) and
        wp > min(0, wc) and wp < max(0, wc)):
        return True
    else:
        return False

In [310]:
point_in_lattice(bi_bonded.structure.lattice.matrix, bi_bonded.structure.cart_coords[0])

True

In [144]:
def get_image(lattice, point, tol=1e-8, verbose=False):
    u = np.cross(lattice[1], lattice[2])
    v = np.cross(lattice[0], lattice[2])
    w = np.cross(lattice[0], lattice[1])
    
    up = np.dot(u, point)
    vp = np.dot(v, point)
    wp = np.dot(w, point)
    
    ua = np.dot(u, lattice[0])
    vb = np.dot(v, lattice[1])
    wc = np.dot(w, lattice[2])
    
    div_a = up/ua 
    div_b = vp/vb 
    div_c = wp/wc
    
#     div_a += tol if div_a < tol else -tol
#     div_b += tol if div_b < tol else -tol
#     div_c += tol if div_c < tol else -tol
    
    if verbose:
        print("div_a: {}\tdiv_b: {}\tdiv_c: {}".format(div_a, div_b, div_c))
    
    image_a = int(np.floor(div_a)) if div_a > 0 else int(np.floor(div_a))
    image_b = int(np.floor(div_b)) if div_b > 0 else int(np.floor(div_b))
    image_c = int(np.floor(div_c)) if div_c > 0 else int(np.floor(div_c))
    
    return (image_a, image_b, image_c)

In [611]:
bi_super = bi * (-1, -2, -2)
lattice = bi.lattice.matrix
point = bi_super.cart_coords[0]

print("lattice:\n\n{}\n".format(lattice))
print("point before: {}".format([float("{:.2f}".format(x)) for x in point]))

# point[0] += 10

print("point after: {}".format([float("{:.2f}".format(x)) for x in point]))

print("image: {}".format(get_image(lattice, point)))

lattice:

[[7.17107425 0.14781153 5.31350797]
 [2.75748514 6.62136166 5.31350793]
 [0.21679984 0.1478116  8.92248119]]

point before: [4.53, -4.73, -2.83]
point after: [4.53, -4.73, -2.83]
image: (0, -1, -1)


### Second fragment finger attempt

In [151]:
from itertools import combinations
def calc_dimensionality(list_images, verbose=False):
    directions = (
        ((0, 1, 1), (2, 1, 1)),  # along x
        ((1, 0, 1), (1, 2, 1)),  # along y
        ((1, 1, 0), (1, 1, 2)),  # along z
        
        ((0, 0, 1), (2, 2, 1)),  # along xy  
        ((0, 1, 0), (2, 1, 2)),  # along xz
        ((1, 0, 0), (1, 2, 2)),  # along yz
        
        ((0, 0, 0), (2, 2, 2)),   # along xyz
    )
    
    spans = [x in list_images and y in list_images
             for x, y in directions]
    n_spans = spans.count(True)
#     print(list_images)
    
    def reverse(image):
        r = []
        for i in image:
            if i == 1:
                r.append(1)
            elif i == 0:
                r.append(2)
            else:
                r.append(0)
        r = tuple(r)
        if r == image:
            return False
        else:
            return r
        
    n_reversed = len([x for x in list_images
                      if lookup[x] is not None and lookup[x] in list_images]) / 2

    # get number of corner images
    # if more than 2 then cannot be 1D
    n_corners = len([x for x in list_images
                     if (2 in x or 0 in x) and 1 not in x])
    if verbose:
        print("n_corners: {}\tn_spans: {}\tn_reversed: {}".format(n_corners, n_spans, n_reversed))
#     if n_corners == 8:
#         dimensionality = 3
#     elif n_corners > 2 or n_spans > 1:
#         dimensionality = 2
#     elif n_spans > 0:
#         dimensionality = 1
#     else:
#         dimensionality = 0

    if n_reversed > 5:
        dimensionality = 3
    elif n_reversed > 1:
        dimensionality = 2
    elif n_reversed == 1:
        dimensionality = 1
    else:
        dimensionality = 0
    return dimensionality

def get_dimensionality(structure, verbose=False):

    sites_to_translate = [i for i, site in enumerate(structure)
                          if 0 in site.frac_coords or 1 in site.frac_coords]
#     print(sites_to_translate)
    structure.translate_sites(sites_to_translate, [1e-8, 1e-8, 1e-8], frac_coords=False)
    bonded_struct = cnn.get_bonded_structure(structure)
    supercell = bonded_struct * (3, 3, 3)

    set_of_visited_sites = set()
    all_visited_sites = set()
    fragments = []
    for i, site in enumerate(supercell.structure):
        visited_sites = set()
        
        def get_connected_sites(site_id):
            for site in supercell.get_connected_sites(site_id):
                coords = tuple(site.site.coords)
                if (coords not in visited_sites and 
                        coords not in all_visited_sites and
                        site.jimage == (0, 0, 0)): 
                    
                    visited_sites.add(coords)
                    all_visited_sites.add(coords)
                    get_connected_sites(site.index)

        get_connected_sites(i)
        site_images = [get_image(structure.lattice.matrix, site)
                       for site in visited_sites]
        
        # if the fragment is bonded to the central image, store the fragment
        if (any([image == (1, 1, 1) for image in site_images]) and
            frozenset(visited_sites) not in set_of_visited_sites):
            
            set_of_visited_sites.add(frozenset(visited_sites))
            
            dimensionality = calc_dimensionality(set(site_images), verbose=verbose)
            
            fragments.append({'atoms': frozenset(visited_sites),
                              'dimensionality': dimensionality})


    n_atoms = [len(fragment['atoms']) for fragment in fragments]
    dimen = [fragment['dimensionality'] for fragment in fragments]
    
    if verbose:
        print("{} groups of connected sites".format(len(set_of_visited_sites)))
        print("the sets have: {} atoms\nwith dimensionality: {}".format(n_atoms, dimen))
        
    return max(dimen), fragments

# get_dimensionality(catio, verbose=True)


In [153]:
dimen , fragments = get_dimensionality(bi)

In [143]:
def dimen(mp_id):
    mpr = MPRester()
    data = mpr.get_entry_by_material_id(mp_id, inc_structure='final')
    struct = data.structure
    return get_dimensionality(struct, verbose=True)
    
dimen('mp-11227')

NameError: name 'get_image' is not defined

Works much better. Now lets compare to the other dimensionality to calculator and see what changes.

### Compare dimensionality methods

In [539]:
from matminer.datasets import load_dataset, available_datasets

df = load_dataset('elastic_tensor_2015')

Fetching elastic_tensor_2015.json.gz from https://ndownloader.figshare.com/files/13220603 to /Users/alexganose/dev/matminer/matminer/datasets/elastic_tensor_2015.json.gz


In [681]:
from matminer.featurizers.base import BaseFeaturizer

class NewDimensionality(BaseFeaturizer):

    def featurize(self, s):
        return [get_dimensionality(s)]

    def feature_labels(self):
        return ["new_dimensionality_five"]

    def citations(self):
        return ["me"]
    
    def implementors(self):
        return ["me"]

In [675]:
from matminer.featurizers.structure import Dimensionality

X = df
d = Dimensionality()
X = d.featurize_dataframe(X, 'structure')


In [682]:
nd = NewDimensionality()
# X = X.drop(columns=['NewDimensionality Exceptions'])

X = nd.featurize_dataframe(X, 'structure', return_errors=True, ignore_errors=True)

In [697]:
X[X['dimensionality'] != X['new_dimensionality_five']][X['dimensionality'] == 2]

/Users/alexganose/miniconda3/envs/common_env3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,material_id,formula,nsites,space_group,volume,structure,elastic_anisotropy,G_Reuss,G_VRH,G_Voigt,K_Reuss,K_VRH,K_Voigt,poisson_ratio,compliance_tensor,elastic_tensor,elastic_tensor_original,dimensionality,new_dimensionality_five,NewDimensionality Exceptions
220,mp-12693,Mg2Rh,6,139,104.037774,"[[1.610584 1.610584 1.44491864] Mg, [1.610...",2.652374,32.308637,39.872414,47.436190,56.677736,65.498864,74.319991,0.246969,"[[0.010921486472281, -0.000591462431469, -0.00...","[[142.66357192180826, 55.80244217677537, 53.76...","[[142.66357192180826, 55.80244217677537, 53.76...",2,0,NaN
294,mp-140,Ga,2,139,37.905223,"[[0. 0. 0.] Ga, [1.449379 1.449379 2.255512] Ga]",2.167179,5.214681,6.344549,7.474418,49.354685,49.366337,49.377988,0.438380,"[[0.030860676289136003, -0.001764103354717, -0...","[[66.5429709675885, 35.89134302865806, 45.0049...","[[66.5429709675885, 35.89134302865806, 45.0049...",2,3,NaN
339,mp-16245,CaSbAu,6,194,157.686141,"[[0. 0. 0.] Ca, [0. 0. 4.1252335...",1.019603,22.144912,23.743349,25.341785,46.396264,53.304561,60.212859,0.306079,"[[0.011809009603428, -0.0064479312355170005, -...","[[127.69461808494853, 72.7215713935361, 20.715...","[[127.69461808494853, 72.7215713935361, 20.715...",2,3,NaN
341,mp-16266,CaZnSi,6,194,128.270974,"[[0. 0. 4.126327] Ca, [0. 0. 0.] C...",0.824510,39.832144,42.373808,44.915471,50.066139,54.732709,59.399280,0.192307,"[[0.007506284772812, -0.000896365888868, -0.00...","[[149.36744442935046, 29.97390686136147, 29.50...","[[149.36744442935046, 29.97390686136147, 29.50...",2,3,NaN
486,mp-2097,SnO,4,129,75.166961,"[[0. 1.933476 1.16286876] Sn, [1.933...",7.265000,13.135000,21.395000,29.655000,21.767000,32.392000,43.017000,0.229000,"[[0.05271271168582901, -0.046234409654551, -0....","[[86.106, 76.0015, 9.675, 0.0, 0.0, -0.0005], ...","[[86.106, 76.0015, 9.675, 0.0, 0.0, -0.0005], ...",2,0,NaN
503,mp-2125,TeO2,24,61,393.775151,"[[0.69678499 4.70779903 2.95420715] Te, [2.175...",1.931000,14.602000,16.941000,19.280000,19.365000,22.551000,25.736000,0.200000,"[[0.015380100815300002, -0.0026816696033420003...","[[81.915, 11.717, 23.0425, 0.0, -0.001, -0.077...","[[81.915, 11.717, 23.0425, 0.0, -0.001, -0.077...",2,0,NaN
633,mp-2646,Mg3Sb2,5,164,133.512590,"[[0. 0. 0.] Mg, [-2.63357700e-06 2.65609333e+...",1.063509,15.922373,17.607785,19.293197,42.103126,42.208190,42.313254,0.316881,"[[0.020453267212745003, -0.010408060642223, -0...","[[74.04919194479548, 36.82562821430389, 20.629...","[[74.04919194479548, 36.82562821430389, 20.629...",2,3,NaN
649,mp-2700,CaSi2,3,166,67.663859,"[[0. 0. 0.] Ca, [2.22157302e-17 1.96978699e-17...",0.090051,46.369906,46.777262,47.184618,62.145034,62.213464,62.281894,0.199398,"[[0.008411557715212, -0.0021034852777220003, -...","[[131.72220435300082, 35.7270423486775, 23.924...","[[131.72220435300082, 35.72704235436183, 23.92...",2,3,NaN
667,mp-279,LiIr,2,187,27.212946,[[-1.6682140e-06 1.5413881e+00 2.2042965e+00...,3.967429,47.327541,65.122168,82.916796,130.348477,143.874801,157.401124,0.303354,"[[0.0032544967170280003, -0.001442720751618, -...","[[385.13383482808024, 172.23773704296713, 23.7...","[[385.13383482808024, 172.23773704296713, 23.7...",2,3,NaN
788,mp-3173,CaAlSi,3,187,67.144580,"[[0. 0. 0.] Ca, [2.10481844 1.21521478 2.18757...",0.429971,38.918685,39.802280,40.685874,51.833741,57.093192,62.352644,0.217155,"[[0.008426982986027001, -0.003448640429062, -0...","[[146.5105726421792, 62.14753518870222, 17.463...","[[146.5105726421792, 62.14753518870222, 17.463...",2,3,NaN


In [149]:
def reverse(image):
    r = []
    for i in image:
        if i == 1:
            r.append(1)
        elif i == 0:
            r.append(2)
        else:
            r.append(0)
    r = tuple(r)
    if r == image:
        return False
    else:
        return r

reverse((0, 0, 0))

(2, 2, 2)

In [584]:
get_dimensionality(fake_struct, verbose=True)

n_corners: 2	n_spans: 1	n_reversed: 1.0
1 groups of connected sites
the sets have: [15] atoms
with dimensionality: [1]


1

NameError: name 'list_images' is not defined

In [595]:
%timeit [reverse((x, y, z)) for x, y, z in product(range(3), range(3), range(3))

17.3 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [150]:
from itertools import product
lookup = {}
for image in product(range(3), range(3), range(3)):
    if image != reverse(image):
        lookup[image] = reverse(image)
    else:
        lookup[image] = False
lookup[(2, 0, 0)] = (0, 2, 2)
lookup[(2, 2, 0)] = (0, 0, 2)
lookup[(0, 2, 0)] = (2, 0, 2)

In [599]:
%timeit [lookup[(x, y, z)] for x, y, z in product(range(3), range(3), range(3))]

3.76 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [629]:
tol = 1e-8
sites_to_translate = [i for i, site in enumerate(bi)
                      if 0 in site.frac_coords]
bi.translate_sites(sites_to_translate, [tol, tol, tol], frac_coords=False)

In [662]:
X.iloc[60]['structure']

Structure Summary
Lattice
    abc : 5.703198000000088 5.703198 5.703198000000088
 angles : 90.0 90.00002009250935 90.0
 volume : 185.50488397788934
      A : 5.703198 0.0 -1e-06
      B : 0.0 5.703198 -0.0
      C : -1e-06 -0.0 5.703198
PeriodicSite: Al (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Al (5.7032, 2.8516, 2.8516) [1.0000, 0.5000, 0.5000]
PeriodicSite: Al (2.8516, 5.7032, 2.8516) [0.5000, 1.0000, 0.5000]
PeriodicSite: Al (2.8516, 2.8516, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Fe (2.8516, 5.7032, 5.7032) [0.5000, 1.0000, 1.0000]
PeriodicSite: Fe (2.8516, 2.8516, 2.8516) [0.5000, 0.5000, 0.5000]
PeriodicSite: Fe (5.7032, 5.7032, 2.8516) [1.0000, 1.0000, 0.5000]
PeriodicSite: Fe (5.7032, 2.8516, 5.7032) [1.0000, 0.5000, 1.0000]
PeriodicSite: Co (4.2774, 1.4258, 1.4258) [0.7500, 0.2500, 0.2500]
PeriodicSite: Co (1.4258, 4.2774, 4.2774) [0.2500, 0.7500, 0.7500]
PeriodicSite: Co (4.2774, 4.2774, 4.2774) [0.7500, 0.7500, 0.7500]
PeriodicSite: Co (1.4258, 

In [693]:
aw = cnn.get_bonded_structure(X.iloc[730]['structure'])
aw

Structure Graph
Structure: 
Structure Summary
Lattice
    abc : 7.1426565686395 7.142656467549591 7.1426565686395
 angles : 38.6072412886225 38.60724490331259 38.6072412886225
 volume : 127.50029879468154
      A : 2.361177 -1.363226 6.601818
      B : 0.0 2.726452 6.601818
      C : -2.361177 -1.363226 6.601818
PeriodicSite: Li (0.0000, 0.0000, 9.9027) [0.5000, 0.5000, 0.5000]
PeriodicSite: Li (0.0000, 0.0000, 7.0115) [0.3540, 0.3540, 0.3540]
PeriodicSite: Li (0.0000, 0.0000, 12.7939) [0.6460, 0.6460, 0.6460]
PeriodicSite: Li (0.0000, 0.0000, 4.2369) [0.2139, 0.2139, 0.2139]
PeriodicSite: Li (0.0000, -0.0000, 15.5686) [0.7861, 0.7861, 0.7861]
PeriodicSite: Sn (-0.0000, -0.0000, 1.4607) [0.0737, 0.0737, 0.0737]
PeriodicSite: Sn (-0.0000, 0.0000, 18.3448) [0.9263, 0.9263, 0.9263]
Graph: bonds
from    to  to_image      weight
----  ----  ------------  ------------------
   0     1  (0, 0, 0)     1.000e+00
   0     2  (0, 0, 0)     1.000e+00
   0     3  (0, 1, 0)     1.000e+00
   0     3 

In [694]:
aw.get_connected_sites(0)

[ConnectedSite(site=PeriodicSite: Li (0.0000, -2.7265, 8.9667) [0.7861, -0.2139, 0.7861], jimage=(0, -1, 0), index=4, weight=1, dist=2.8826415952143187),
 ConnectedSite(site=PeriodicSite: Li (0.0000, 2.7265, 10.8387) [0.2139, 1.2139, 0.2139], jimage=(0, 1, 0), index=3, weight=1, dist=2.8826415952143196),
 ConnectedSite(site=PeriodicSite: Li (2.3612, 1.3632, 8.9667) [0.7861, 0.7861, -0.2139], jimage=(0, 0, -1), index=4, weight=1, dist=2.88264184569654),
 ConnectedSite(site=PeriodicSite: Li (-2.3612, 1.3632, 8.9667) [-0.2139, 0.7861, 0.7861], jimage=(-1, 0, 0), index=4, weight=1, dist=2.88264184569654),
 ConnectedSite(site=PeriodicSite: Li (-2.3612, -1.3632, 10.8387) [0.2139, 0.2139, 1.2139], jimage=(0, 0, 1), index=3, weight=1, dist=2.8826418456965404),
 ConnectedSite(site=PeriodicSite: Li (2.3612, -1.3632, 10.8387) [1.2139, 0.2139, 0.2139], jimage=(1, 0, 0), index=3, weight=1, dist=2.882641845696541),
 ConnectedSite(site=PeriodicSite: Li (0.0000, 0.0000, 12.7939) [0.6460, 0.6460, 0.646

In [133]:
from itertools import combinations

def reverse(image):
    r = []
    for i in image:
        if i == 1:
            r.append(1)
        elif i == 0:
            r.append(2)
        else:
            r.append(0)
    r = tuple(r)
    if r == image:
        return False
    else:
        return r
    
reversals = 

def calc_dimensionality(list_images, verbose=False):
    n_reversed = len([x for x in list_images
                      if lookup[x] is not None and lookup[x] in list_images]) / 2
    if verbose:
        print("n_reversed: {}".format(n_reversed))

    if n_reversed > 5:
        dimensionality = 3
    elif n_reversed > 1:
        dimensionality = 2
    elif n_reversed == 1:
        dimensionality = 1
    else:
        dimensionality = 0
    return dimensionality

def get_dimensionality(structure, verbose=False):

    sites_to_translate = [i for i, site in enumerate(structure)
                          if 0 in site.frac_coords or 1 in site.frac_coords]
    structure.translate_sites(sites_to_translate, [1e-8, 1e-8, 1e-8], frac_coords=False)
    bonded_struct = cnn.get_bonded_structure(structure)
    supercell = bonded_struct * (3, 3, 3)

    all_visited_sites = set()
    fragments = []
    
    for i, site in enumerate(supercell.structure):
        fragment = []
        
        def get_connected_sites(site_id):
            for site in supercell.get_connected_sites(site_id):
                coords = tuple(site.site.coords)
                
                if (coords not in all_visited_sites and
                        site.jimage == (0, 0, 0)):     
                    fragment.append(coords)
                    all_visited_sites.add(coords)
                    get_connected_sites(site.index)

        get_connected_sites(i)
        
        site_images = [get_image(structure.lattice.matrix, site)
                       for site in fragment]
        
        # if the fragment is bonded to the central image, store the fragment
        if fragement and any([image == (1, 1, 1) for image in site_images]):
            
            dimensionality = calc_dimensionality(set(site_images), verbose=verbose)
            fragments.append({'atoms': fragment,
                              'dimensionality': dimensionality})

    dimen = [fragment['dimensionality'] for fragment in fragments]
    
    if verbose:
        n_atoms = [len(fragment['atoms']) for fragment in fragments]
        print("{} groups of connected sites".format(len(set_of_visited_sites)))
        print("the sets have: {} atoms\nwith dimensionality: {}".format(n_atoms, dimen))
        
    return max(dimen)

SyntaxError: invalid syntax (<ipython-input-133-94de09b1a9b0>, line 18)

In [56]:
from pymatgen.core.structure import Structure
from pymatgen.core.surface import miller_index_from_sites


structure = Structure(
    [10, 0, 0, 0, 10, 0, 0, 0, 10],
    ['Fe', 'Fe', 'Fe'],
    [[0.5, 0.8, 0.8], [0.5, 0.6, 0.6], [0.5, 0.3, 0.3]]
)

miller_index_from_sites(structure.lattice.matrix, structure.cart_coords)

array([nan, nan, nan])

In [167]:
import numpy as np
from pymatgen.core.surface import get_integer_index

def fit_plane_to_points(lattice, points):
    points = np.asarray(points)
    G = points.sum(axis=0) / points.shape[0]

    # run SVD
    u, s, vh = np.linalg.svd(points - G)

    # unitary normal vector
    u_norm = vh[2, :]
    print(u_norm)
    return cart_vector_to_lattice_vector(lattice, u_norm)

def cart_vector_to_lattice_vector(lattice, vector):
    M = np.array([
        [lattice.a * np.sin(np.deg2rad(lattice.beta)), lattice.b * np.sin(np.deg2rad(lattice.alpha)) * np.cos(np.deg2rad(lattice.gamma)), 0],
        [0, lattice.b * np.sin(np.deg2rad(lattice.alpha)) * np.sin(np.deg2rad(lattice.gamma)), 0],
        [lattice.a * np.cos(np.deg2rad(lattice.beta)), lattice.b * np.cos(np.deg2rad(lattice.alpha)), lattice.c]
    ])
    vec = np.dot(np.linalg.inv(M), vector)
    print(vec)
    return get_integer_index(vec)
    
fit_plane_to_points(structure.lattice, structure.cart_coords)

[1. 0. 0.]
[ 1.000000e-01  0.000000e+00 -6.123234e-18]


array([ 1.000000e+00,  0.000000e+00, -6.123234e-17])

In [171]:
fit_plane_to_points((bi * (3, 3, 3)).lattice, [list(x) for x in fragments[0]['atoms']])

[0.4394625  0.29962028 0.84681781]
[0.00974918 0.01798719 0.01457038]


/Users/alexganose/dev/pymatgen/pymatgen/core/surface.py:1693: RuntimeWarning:

divide by zero encountered in true_divide



array([inf, inf, inf])

In [164]:
len([list(x) for x in fragments[0]['atoms']])

52

In [81]:
1e+1

10.0

In [46]:
import numpy as np
import scipy.optimize

def fitPLaneLTSQ(XYZ):
    # Fits a plane to a point cloud, 
    # Where Z = aX + bY + c        ----Eqn #1
    # Rearanging Eqn1: aX + bY -Z +c =0
    # Gives normal (a,b,-1)
    # Normal = (a,b,-1)
    [rows,cols] = XYZ.shape
    G = np.ones((rows,3))
    G[:,0] = XYZ[:,0]  #X
    G[:,1] = XYZ[:,1]  #Y
    Z = XYZ[:,2]
    (a,b,c),resid,rank,s = np.linalg.lstsq(G,Z) 
    normal = (a,b,-1)
    nn = np.linalg.norm(normal)
    normal = normal / nn
    return normal


def fitPlaneSVD(XYZ):
    [rows,cols] = XYZ.shape
    # Set up constraint equations of the form  AB = 0,
    # where B is a column vector of the plane coefficients
    # in the form b(1)*X + b(2)*Y +b(3)*Z + b(4) = 0.
    p = (np.ones((rows,1)))
    AB = np.hstack([XYZ,p])
    [u, d, v] = np.linalg.svd(AB,0)        
    B = v[3,:];                    # Solution is last column of v.
    nn = np.linalg.norm(B[0:3])
    B = B / nn
    return B[0:3]


def fitPlaneEigen(XYZ):
    # Works, in this case but don't understand!
    average=sum(XYZ)/XYZ.shape[0]
    covariant=np.cov(XYZ - average)
    eigenvalues,eigenvectors = np.linalg.eig(covariant)
    want_max = eigenvectors[:,eigenvalues.argmax()]
    (c,a,b) = want_max[3:6]    # Do not understand! Why 3:6? Why (c,a,b)?
    normal = np.array([a,b,c])
    nn = np.linalg.norm(normal)
    return normal / nn  

def fitPlaneSolve(XYZ):
    X = XYZ[:,0]
    Y = XYZ[:,1]
    Z = XYZ[:,2] 
    npts = len(X)
    A = np.array([ [sum(X*X), sum(X*Y), sum(X)],
                   [sum(X*Y), sum(Y*Y), sum(Y)],
                   [sum(X),   sum(Y), npts] ])
    B = np.array([ [sum(X*Z), sum(Y*Z), sum(Z)] ])
    normal = np.linalg.solve(A,B.T)
    nn = np.linalg.norm(normal)
    normal = normal / nn
    return normal.ravel()

def fitPlaneOptimize(XYZ):
    def residiuals(parameter,f,x,y):
        return [(f[i] - model(parameter,x[i],y[i])) for i in range(len(f))]


    def model(parameter, x, y):
        a, b, c = parameter
        return a*x + b*y + c

    X = XYZ[:,0]
    Y = XYZ[:,1]
    Z = XYZ[:,2]
    p0 = [1., 1.,1.] # initial guess
    result = scipy.optimize.leastsq(residiuals, p0, args=(Z,X,Y))[0]
    normal = result[0:3]
    nn = np.linalg.norm(normal)
    normal = normal / nn
    return normal


XYZ = np.array([
    [0,0,1],
    [0,1,2],
    [0,2,3],
    [1,0,1],
#     [1,1,2],
#     [1,2,3],
#     [2,0,1],
#     [2,1,2],
#     [2,2,3]
    ])
print("Solve: ", fitPlaneSolve(structure.cart_coords))
print("Optim: ",fitPlaneOptimize(structure.cart_coords))
print("SVD:   ",fitPlaneSVD(structure.cart_coords))
print("LTSQ:  ",fitPLaneLTSQ(structure.cart_coords))
print("Eigen: ",fitPlaneEigen(structure.cart_coords))

Solve:  [1.94731868e-17 7.81898595e-01 6.23405636e-01]
Optim:  [ 0.19638504  0.00175585 -0.98052528]
SVD:    [ 1.00000000e+00 -3.05898024e-16 -2.67948900e-16]
LTSQ:   [ 0.10150853  0.66204204 -0.74256067]


/Users/alexganose/miniconda3/envs/common_env3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  from ipykernel import kernelapp as app


ValueError: not enough values to unpack (expected 3, got 1)

In [33]:
fit_plane_to_points(

array([[0, 0, 1],
       [0, 1, 2],
       [0, 2, 3],
       [1, 0, 1],
       [1, 1, 2],
       [1, 2, 3],
       [2, 0, 1],
       [2, 1, 2],
       [2, 2, 3]])

In [47]:
site = structure[0]

In [50]:
structure.lattice.

TypeError: d_hkl() missing 1 required positional argument: 'miller_index'